# Day 19: Basic Options Strategies

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class3_trading_strategies/week4_options_basics/day19_basic_strategies.ipynb)

---

## Learning Objectives

1. **Learn** the Wheel Strategy (CSP + CC)
2. **Understand** options income strategies
3. **Master** rolling options positions
4. **Build** a systematic options income approach
5. **Manage** option positions through various scenarios

---

## Lecture (30 minutes)

### The Wheel Strategy

```
THE WHEEL STRATEGY
==================

A systematic approach combining:
1. Cash-Secured Puts (CSP)
2. Covered Calls (CC)

THE CYCLE:

     START: Have Cash
           |
           v
    +------------------+
    | Sell Cash-Secured|
    | Put (CSP)        |<---------+
    +------------------+          |
           |                      |
    Expires worthless?            |
    YES: Keep premium,            |
         repeat CSP -------->-----+
           |                      |
    NO: Assigned                  |
           |                      |
           v                      |
    +------------------+          |
    | Now Own Shares   |          |
    +------------------+          |
           |                      |
           v                      |
    +------------------+          |
    | Sell Covered     |          |
    | Call (CC)        |          |
    +------------------+          |
           |                      |
    Called away?                  |
    YES: Back to cash,            |
         start CSP again --->-----+
           |                      
    NO: Keep premium,             
        repeat CC                 
```

### Wheel Strategy Example

```
WHEEL EXAMPLE: AAPL
===================

MONTH 1: CSP Phase
- AAPL at $175
- Sell $170 Put for $2.00
- Hold $17,000 cash
- Stock stays at $175
- Put expires worthless
- PROFIT: $200

MONTH 2: CSP Phase (repeat)
- AAPL at $172
- Sell $165 Put for $2.50
- Stock drops to $160
- ASSIGNED: Buy 100 shares at $165
- Effective cost: $165 - $2.50 = $162.50
- PROFIT: $250 (but now own shares)

MONTH 3: CC Phase
- Own shares at $162.50
- AAPL at $160
- Sell $170 Call for $1.50
- Stock rises to $172
- CALLED AWAY at $170
- Gain: ($170 - $162.50 + $1.50) × 100 = $900

TOTAL: $200 + $250 + $900 = $1,350 in 3 months
Return: $1,350 / $17,000 = 7.9% (3 months)
Annualized: ~32%
```

### Options Income Mechanics

```
OPTIONS INCOME FUNDAMENTALS
===========================

WHY SELL OPTIONS?
1. Time decay (theta) works FOR you
2. Probability of profit typically >50%
3. Defined, recurring income
4. Flexibility in strike selection

INCOME POTENTIAL:
- CSP: 1-3% monthly on cash
- CC: 1-2% monthly on stock
- Combined: 12-30% annualized

RISKS:
- Stock drops significantly (CSP)
- Miss big upside (CC)
- Getting stuck in losing position

IDEAL WHEEL CANDIDATES:
- Stocks you want to own
- Quality companies
- Moderate volatility (not too wild)
- Liquid options markets
- No near-term catalysts
```

### Rolling Options

```
ROLLING OPTIONS POSITIONS
=========================

WHAT IS ROLLING?
Closing current option and opening new one
= Buy to close + Sell to open

ROLL TYPES:

ROLL OUT (same strike, later date):
- When near expiration
- Stock near your strike
- Collect more premium

ROLL UP (higher strike, same/later date):
- Stock moved against you (CC)
- Take profit on current, reset higher
- Usually costs money (debit)

ROLL DOWN (lower strike, same/later date):
- Stock dropped (CSP)
- Lower strike to avoid assignment
- Usually receive credit

ROLL OUT AND UP/DOWN:
- Combine time and strike change
- More flexibility

ROLL FOR CREDIT RULE:
Only roll if you receive net credit
(Don't pay to avoid assignment)
```

### Position Management

```
MANAGING THE WHEEL
==================

CSP MANAGEMENT:

Stock Rising (good):
- Let put expire worthless
- Sell new put next cycle

Stock Falling:
Option 1: Accept assignment
  - Buy shares at strike
  - Start selling covered calls
Option 2: Roll down and out
  - Buy back put
  - Sell lower strike, further out
Option 3: Close for loss
  - If thesis changed

CC MANAGEMENT:

Stock Falling:
- Keep premium
- Sell new call next cycle
- Wait for stock to recover

Stock Rising:
Option 1: Let shares be called
  - Take profit
  - Start CSP again
Option 2: Roll up and out
  - Buy back call
  - Sell higher strike, further out
Option 3: Buy back call (for loss)
  - Keep shares, no call

GOLDEN RULES:
1. Only trade on stocks you want to own
2. Size appropriately (25-30% cash per position)
3. Roll for credit only
4. Accept assignment as opportunity
```

---

## Hands-On Practice (15 minutes)

In [ ]:
!pip install yfinance pandas numpy matplotlib -q
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print("Libraries loaded!")

In [ ]:
def wheel_strategy_planner(ticker, capital=50000):
    """Plan wheel strategy for a stock."""
    stock = yf.Ticker(ticker)
    info = stock.info
    current_price = info.get('currentPrice', info.get('regularMarketPrice', 0))
    
    print(f"\n{'='*70}")
    print(f"WHEEL STRATEGY PLANNER: {ticker}")
    print(f"{'='*70}")
    print(f"\nCurrent Price: ${current_price:.2f}")
    print(f"Capital Available: ${capital:,}")
    
    # Calculate how many contracts we can sell
    max_contracts = int(capital / (current_price * 100))
    shares_controlled = max_contracts * 100
    capital_per_contract = current_price * 100
    
    print(f"\n{'-'*70}")
    print("POSITION SIZING:")
    print(f"{'-'*70}")
    print(f"Max Contracts: {max_contracts}")
    print(f"Shares Controlled: {shares_controlled}")
    print(f"Capital per Contract: ${capital_per_contract:,.2f}")
    
    # Recommended sizing (25% of capital max)
    recommended = int(capital * 0.25 / (current_price * 100))
    print(f"\nRecommended (25% max): {max(1, recommended)} contracts")
    
    # Get options data
    try:
        expirations = stock.options
        if len(expirations) > 0:
            exp = expirations[0]  # Nearest expiration
            options = stock.option_chain(exp)
            
            # CSP recommendations
            puts = options.puts
            otm_puts = puts[
                (puts['strike'] < current_price * 0.95) & 
                (puts['strike'] > current_price * 0.85) &
                (puts['bid'] > 0)
            ]
            
            print(f"\n{'-'*70}")
            print(f"CSP RECOMMENDATIONS (Exp: {exp}):")
            print(f"{'-'*70}")
            
            for _, row in otm_puts.head(3).iterrows():
                strike = row['strike']
                premium = row['bid']
                monthly_return = premium / strike * 100
                effective_buy = strike - premium
                
                print(f"\n  Strike: ${strike:.2f}")
                print(f"  Premium: ${premium:.2f} (${premium * 100:.0f}/contract)")
                print(f"  Monthly Yield: {monthly_return:.2f}%")
                print(f"  If assigned, buy at: ${effective_buy:.2f}")
            
            # CC recommendations (if we owned shares)
            calls = options.calls
            otm_calls = calls[
                (calls['strike'] > current_price * 1.03) & 
                (calls['strike'] < current_price * 1.15) &
                (calls['bid'] > 0)
            ]
            
            print(f"\n{'-'*70}")
            print(f"COVERED CALL RECOMMENDATIONS (If Assigned):")
            print(f"{'-'*70}")
            
            for _, row in otm_calls.head(3).iterrows():
                strike = row['strike']
                premium = row['bid']
                monthly_return = premium / current_price * 100
                upside = (strike - current_price) / current_price * 100
                
                print(f"\n  Strike: ${strike:.2f} ({upside:+.1f}% upside)")
                print(f"  Premium: ${premium:.2f} (${premium * 100:.0f}/contract)")
                print(f"  Monthly Yield: {monthly_return:.2f}%")
    except Exception as e:
        print(f"\nCould not fetch options data: {e}")
    
    return current_price, max_contracts

wheel_strategy_planner('AAPL', capital=50000)

In [ ]:
def simulate_wheel_trades(starting_capital=50000, months=12):
    """Simulate wheel strategy performance."""
    np.random.seed(42)
    
    # Simulation parameters
    stock_price = 175
    monthly_volatility = 0.06  # 6% monthly moves
    put_strike_pct = 0.95  # 5% OTM
    call_strike_pct = 1.05  # 5% OTM
    avg_put_premium = 0.015  # 1.5% of strike
    avg_call_premium = 0.012  # 1.2% of strike
    
    # State tracking
    capital = starting_capital
    shares_owned = 0
    cost_basis = 0
    total_premium = 0
    
    trades = []
    
    print(f"\n{'='*70}")
    print("WHEEL STRATEGY SIMULATION")
    print(f"{'='*70}")
    print(f"Starting Capital: ${starting_capital:,}")
    print(f"Simulation Period: {months} months\n")
    
    for month in range(1, months + 1):
        # Stock price random walk
        stock_return = np.random.normal(0.005, monthly_volatility)
        new_price = stock_price * (1 + stock_return)
        
        trade = {'Month': month, 'Stock Price': new_price}
        
        if shares_owned == 0:
            # CSP Phase
            contracts = int(capital / (stock_price * 100))
            if contracts == 0:
                contracts = 1
            
            put_strike = stock_price * put_strike_pct
            premium = put_strike * avg_put_premium * 100 * contracts
            
            if new_price < put_strike:
                # Assigned
                shares_owned = contracts * 100
                cost_basis = put_strike - (premium / shares_owned)
                capital -= put_strike * shares_owned
                capital += premium
                trade['Action'] = f'CSP ASSIGNED'
                trade['Strike'] = put_strike
                trade['Shares'] = shares_owned
            else:
                # Expired worthless
                total_premium += premium
                capital += premium
                trade['Action'] = 'CSP EXPIRED'
                trade['Premium'] = premium
        else:
            # CC Phase
            contracts = shares_owned // 100
            call_strike = stock_price * call_strike_pct
            premium = call_strike * avg_call_premium * 100 * contracts
            
            if new_price > call_strike:
                # Called away
                proceeds = call_strike * shares_owned
                gain = (call_strike - cost_basis) * shares_owned + premium
                capital += proceeds + premium
                total_premium += premium
                trade['Action'] = f'CC CALLED AWAY'
                trade['Strike'] = call_strike
                trade['Gain'] = gain
                shares_owned = 0
            else:
                # Keep shares
                total_premium += premium
                capital += premium
                trade['Action'] = 'CC EXPIRED'
                trade['Premium'] = premium
        
        stock_price = new_price
        
        # Portfolio value
        portfolio_value = capital + (shares_owned * stock_price)
        trade['Portfolio'] = portfolio_value
        trades.append(trade)
    
    # Results
    final_value = capital + (shares_owned * stock_price)
    total_return = (final_value - starting_capital) / starting_capital * 100
    
    print(f"{'-'*70}")
    print("TRADE LOG:")
    print(f"{'-'*70}")
    
    df = pd.DataFrame(trades)
    for _, row in df.iterrows():
        action = row.get('Action', '')
        print(f"Month {row['Month']:2}: Stock ${row['Stock Price']:.2f} | {action} | Value: ${row['Portfolio']:,.0f}")
    
    print(f"\n{'-'*70}")
    print("SIMULATION RESULTS:")
    print(f"{'-'*70}")
    print(f"Starting Capital: ${starting_capital:,}")
    print(f"Final Portfolio Value: ${final_value:,.2f}")
    print(f"Total Premium Collected: ${total_premium:,.2f}")
    print(f"Total Return: {total_return:.1f}%")
    print(f"Annualized Return: {total_return * 12 / months:.1f}%")
    
    if shares_owned > 0:
        print(f"\nCurrently Holding: {shares_owned} shares at ${cost_basis:.2f}")
    
    return df

trades_df = simulate_wheel_trades()

In [ ]:
def rolling_options_calculator(current_option_price, new_option_price, contracts=1):
    """Calculate cost/credit of rolling an options position."""
    shares = contracts * 100
    
    # Buy to close current
    close_cost = current_option_price * shares
    
    # Sell to open new
    open_credit = new_option_price * shares
    
    # Net
    net = open_credit - close_cost
    
    print(f"\n{'='*50}")
    print("ROLL OPTIONS CALCULATOR")
    print(f"{'='*50}")
    print(f"\nContracts: {contracts}")
    print(f"\nBuy to Close Current: ${current_option_price:.2f} × {shares} = ${close_cost:,.2f}")
    print(f"Sell to Open New: ${new_option_price:.2f} × {shares} = ${open_credit:,.2f}")
    
    if net > 0:
        print(f"\nNET CREDIT: ${net:,.2f}")
        print("Recommendation: ROLL (you receive money)")
    elif net < 0:
        print(f"\nNET DEBIT: ${abs(net):,.2f}")
        print("Recommendation: Consider alternatives (rolling costs money)")
    else:
        print(f"\nEVEN: $0")
    
    return net

# Example: Rolling a CSP down and out
print("Example: Stock fell, rolling CSP down and out")
print("Current: $170 Put expiring this week, trading at $3.50")
print("New: $165 Put expiring next month, bid at $4.00")
rolling_options_calculator(current_option_price=3.50, new_option_price=4.00, contracts=2)

In [ ]:
def find_wheel_candidates():
    """Find good wheel strategy candidates."""
    # Quality large-cap stocks with options
    candidates = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'JNJ', 'PG', 'KO', 'HD', 'V', 'MA']
    
    results = []
    
    print(f"\n{'='*80}")
    print("WHEEL STRATEGY CANDIDATES")
    print(f"{'='*80}\n")
    
    for ticker in candidates:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            current_price = info.get('currentPrice', info.get('regularMarketPrice', 0))
            pe = info.get('trailingPE', 0)
            div_yield = info.get('dividendYield', 0)
            
            # Get options for premium estimate
            expirations = stock.options
            if len(expirations) == 0:
                continue
                
            options = stock.option_chain(expirations[0])
            puts = options.puts
            
            # Find 5% OTM put
            target_strike = current_price * 0.95
            close_puts = puts[(puts['strike'] >= target_strike * 0.95) & 
                             (puts['strike'] <= target_strike * 1.05)]
            
            if len(close_puts) > 0:
                put_row = close_puts.iloc[0]
                put_premium = put_row['bid']
                put_strike = put_row['strike']
                monthly_yield = (put_premium / put_strike) * 100
            else:
                continue
            
            results.append({
                'Ticker': ticker,
                'Price': current_price,
                'P/E': pe,
                'Div Yield %': div_yield * 100 if div_yield else 0,
                'Put Strike': put_strike,
                'Put Premium': put_premium,
                'Monthly Yield %': monthly_yield,
                'Ann. Yield %': monthly_yield * 12
            })
            
        except Exception as e:
            pass
    
    if results:
        df = pd.DataFrame(results).sort_values('Monthly Yield %', ascending=False)
        print(df.round(2).to_string(index=False))
        
        print(f"\n{'-'*80}")
        print("NOTES:")
        print("- Higher yield = higher risk (more volatile or close to strike)")
        print("- Consider P/E and quality when selecting")
        print("- Add dividend yield for total income potential")
    else:
        print("Could not analyze candidates.")
    
    return results

wheel_candidates = find_wheel_candidates()

In [ ]:
def plot_wheel_income(trades_df):
    """Plot wheel strategy performance."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Portfolio value over time
    ax1 = axes[0]
    ax1.plot(trades_df['Month'], trades_df['Portfolio'], 'b-', lw=2, marker='o')
    ax1.axhline(y=trades_df['Portfolio'].iloc[0], color='gray', linestyle='--', label='Starting Capital')
    ax1.set_xlabel('Month')
    ax1.set_ylabel('Portfolio Value ($)')
    ax1.set_title('Wheel Strategy Portfolio Growth')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Monthly returns
    ax2 = axes[1]
    returns = trades_df['Portfolio'].pct_change().fillna(0) * 100
    colors = ['green' if r >= 0 else 'red' for r in returns]
    ax2.bar(trades_df['Month'], returns, color=colors, alpha=0.7)
    ax2.axhline(y=0, color='black', linestyle='-', lw=1)
    ax2.set_xlabel('Month')
    ax2.set_ylabel('Monthly Return (%)')
    ax2.set_title('Monthly Returns')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

if 'trades_df' in dir():
    plot_wheel_income(trades_df)

---

## Quiz

In [ ]:
quiz = [
    {"q": "The Wheel Strategy combines which two strategies?", "a": "C", 
     "opts": ["A) Long calls + long puts", "B) Straddles + strangles", 
              "C) Cash-secured puts + covered calls", "D) Iron condors + butterflies"]},
    {"q": "In the wheel, what happens when a CSP is assigned?", "a": "B",
     "opts": ["A) You lose money", "B) You buy shares at the strike price", 
              "C) You sell shares", "D) Nothing, it expires"]},
    {"q": "Rolling 'out and down' means?", "a": "D",
     "opts": ["A) Higher strike, sooner expiration", "B) Higher strike, later expiration", 
              "C) Lower strike, sooner expiration", "D) Lower strike, later expiration"]},
    {"q": "When should you roll an option?", "a": "A",
     "opts": ["A) When you can receive a net credit", "B) Always roll", 
              "C) Never roll", "D) Only when losing money"]},
    {"q": "Best wheel candidates are?", "a": "C",
     "opts": ["A) Penny stocks", "B) High-volatility meme stocks", 
              "C) Quality stocks you'd want to own", "D) Stocks near bankruptcy"]}
]

def run_quiz():
    for i, q in enumerate(quiz):
        print(f"\nQ{i+1}: {q['q']}")
        for opt in q['opts']:
            print(f"   {opt}")
    print("\n" + "="*50)
    print("ANSWERS: 1-C, 2-B, 3-D, 4-A, 5-C")
    
run_quiz()

---

## Summary

- **Wheel Strategy**: Sell CSP → Get assigned → Sell CC → Repeat
- **Income potential**: 1-3% monthly (12-36% annualized)
- **Rolling**: Close current, open new - ideally for credit
- **Only wheel** stocks you want to own at strike
- **Size appropriately**: Max 25-30% capital per position

**Tomorrow**: Day 20 - Week 4 Review